# PyDI Schema Matching and Data Translation Example

This notebook demonstrates the comprehensive schema matching and data translation capabilities in PyDI.

What this shows:
- Load datasets with provenance tracking
- **Label-based schema matching**: Compare column names using string similarity
- **Instance-based schema matching**: Compare column contents using value distributions
- **Duplicate-based schema matching**: Use known record correspondences to find schema matches
- **Schema mapping evaluation**: Assess matching quality with precision, recall, and F1 scores
- **Data translation**: Apply schema mappings to transform DataFrames
- **End-to-end workflow**: Complete schema alignment pipeline

Run cells below in order. Adjust paths if running outside the repo root.

In [1]:
# PyDI imports
from PyDI.io import load_csv, load_xml, load_fwf
from PyDI.schemamatching import (
    LabelBasedSchemaMatcher,
    InstanceBasedSchemaMatcher, 
    DuplicateBasedSchemaMatcher,
    SchemaMappingEvaluator
)
from PyDI.datatranslation import MappingTranslator

# Additional imports
import pandas as pd
from pathlib import Path

def repo_root():
    """Return the repository root directory."""
    # For notebooks in PyDI/examples/, go up 2 levels to reach repo root
    if '__file__' in globals():
        return Path(__file__).parent.parent.parent
    else:
        # In Jupyter, find the pyproject.toml to locate repo root
        current = Path.cwd()
        while current != current.parent:
            if (current / 'pyproject.toml').exists():
                return current
            current = current.parent
        return Path.cwd()  # fallback

## Step 1: Load datasets with provenance

We'll use the movie datasets as our primary example - a movie list CSV and an actors text file.

In [2]:
root = repo_root()
movies_path = root / "input" / "movies" / "schemamatching" / "data" / "movie_list.csv"
actors_path = root / "input" / "movies" / "schemamatching" / "data" / "actors.txt"

print(f"Movies data: {movies_path}")
print(f"Actors data: {actors_path}")

# Load movie dataset
movies_df = load_csv(movies_path, name="movies")
print(f"\nMovies shape: {movies_df.shape}")
print(f"Movies columns: {list(movies_df.columns)[:10]}...")

# Load actors dataset using PyDI's new load_fwf function
actors_df = load_fwf(actors_path, name="actors",
                    header=None,
                    names=["gender", "id", "year", "name", "movie", "age", "birthplace", "birth_month", "birth_day", "birth_year"],
                    encoding='latin-1')  # Handle special characters

print(f"\nActors shape: {actors_df.shape}")
print(f"Actors columns: {list(actors_df.columns)}")

Movies data: c:\Users\Ralph\dev\pydi\input\movies\schemamatching\data\movie_list.csv
Actors data: c:\Users\Ralph\dev\pydi\input\movies\schemamatching\data\actors.txt

Movies shape: (656, 23)
Movies columns: ['movies_id', 'id', 'year', 'exclude', 'Film', 'Lead Studio', 'Rotten Tomatoes', 'Audience Score', 'Story', 'Genre']...

Actors shape: (156, 11)
Actors columns: ['actors_id', 'gender', 'id', 'year', 'name', 'movie', 'age', 'birthplace', 'birth_month', 'birth_day', 'birth_year']


In [3]:
# Preview the datasets
print("=== Movies Dataset Sample ===")
display(movies_df.head(3))

print("\n=== Actors Dataset Sample ===") 
display(actors_df.head(3))

=== Movies Dataset Sample ===


,movies_id,id,year,exclude,Film,Lead Studio,Rotten Tomatoes,Audience Score,Story,Genre,...,Foreign Gross,Worldwide Gross,Budget,Profit,Proftitability,Opening Weekend,Oscar,Bafta,Source,Column
0,movies-0000,1,2010,NaN,127 Hours,Independent,93.0,84,Escape,Adventure,...,42.4,60.73,18.0,42.73,337.39%,0.26,NaN,NaN,http://boxofficemojo.com/movies/?id=127hours.htm,NaN
1,movies-0001,2,2010,NaN,A Nightmare on Elm Street,Warner Bros.,13.0,40,Monster Force,Horror,...,52.59,115.66,35.0,80.66,330.46%,32.9,NaN,NaN,NaN,NaN
2,movies-0002,3,2010,NaN,Alice in Wonderland,Disney,52.0,72,Journey And Return,Adventure,...,690.2,1024.39,200.0,824.39,512.20%,116.1,NaN,NaN,NaN,NaN



=== Actors Dataset Sample ===


,actors_id,gender,id,year,name,movie,age,birthplace,birth_month,birth_day,birth_year
0,actors-0000,f,1,1929,Janet Gaynor,7th Heaven,22,Pennsylvania,10,6,1906
1,actors-0001,f,2,1930,Mary Pickford,Coquette,37,Canada,4,8,1892
2,actors-0002,f,3,1931,Norma Shearer,The Divorcee,28,Canada,8,10,1902


## Step 2: Label-Based Schema Matching

Label-based matching compares column names using string similarity metrics. This is useful when datasets have similar but not identical column names.

In [4]:
# Initialize label-based matcher with Jaccard similarity
label_matcher = LabelBasedSchemaMatcher(
    similarity_function="jaccard",
    preprocess=str.lower,  # Convert to lowercase for comparison
    tokenize=True  # Split column names into tokens
)

# Find schema correspondences
label_mappings = label_matcher.match(
    source_dataset=movies_df,
    target_dataset=actors_df,
    threshold=0.3  # Lower threshold to see more potential matches
)

print(f"Label-based matching found {len(label_mappings)} correspondences:")
display(label_mappings.sort_values("score", ascending=False))

Label-based matching found 3 correspondences:


,source_dataset,source_column,target_dataset,target_column,score,notes
0,movies,id,actors,id,1.0,similarity_function=jaccard
1,movies,year,actors,year,1.0,similarity_function=jaccard
2,movies,year,actors,birth_year,0.5,similarity_function=jaccard


In [5]:
# Try different similarity functions
similarity_functions = ["levenshtein", "jaro_winkler", "cosine"]

for sim_func in similarity_functions:
    matcher = LabelBasedSchemaMatcher(
        similarity_function=sim_func,
        preprocess=str.lower,
        tokenize=True
    )
    
    mappings = matcher.match(
        source_dataset=movies_df,
        target_dataset=actors_df,
        threshold=0.4
    )
    
    print(f"\n{sim_func.upper()} similarity found {len(mappings)} matches:")
    if not mappings.empty:
        for _, row in mappings.sort_values("score", ascending=False).head(3).iterrows():
            print(f"  {row['source_column']} <-> {row['target_column']} (score: {row['score']:.3f})")
    else:
        print("  No matches found above threshold")


LEVENSHTEIN similarity found 3 matches:
  id <-> id (score: 1.000)
  year <-> year (score: 1.000)
  year <-> birth_year (score: 0.500)

JARO_WINKLER similarity found 2 matches:
  id <-> id (score: 1.000)
  year <-> year (score: 1.000)

COSINE similarity found 3 matches:
  id <-> id (score: 1.000)
  year <-> year (score: 1.000)
  year <-> birth_year (score: 0.707)


## Step 3: Instance-Based Schema Matching

Instance-based matching analyzes the actual values in columns to find correspondences. This is powerful when column names are different but the data content is similar.

In [6]:
# Initialize instance-based matcher
instance_matcher = InstanceBasedSchemaMatcher(
    vector_creation_method="term_frequencies",
    similarity_function="cosine",
    max_sample_size=500,  # Limit samples for performance
    min_non_null_ratio=0.1  # Require at least 10% non-null values
)

# Find schema correspondences using instance-based matching
instance_mappings = instance_matcher.match(
    source_dataset=movies_df,
    target_dataset=actors_df,
    threshold=0.1  # Lower threshold to explore more matches
)

print(f"Instance-based matching found {len(instance_mappings)} correspondences:")
if not instance_mappings.empty:
    display(instance_mappings.sort_values("score", ascending=False).head(10))
else:
    print("No correspondences found above threshold.")

Instance-based matching found 12 correspondences:


,source_dataset,source_column,target_dataset,target_column,score,notes
4,movies,Film,actors,movie,0.912066,"vector_method=term_frequencies,similarity=cosine"
5,movies,Audience Score,actors,id,0.691895,"vector_method=term_frequencies,similarity=cosine"
6,movies,Audience Score,actors,age,0.498533,"vector_method=term_frequencies,similarity=cosine"
0,movies,id,actors,id,0.300942,"vector_method=term_frequencies,similarity=cosine"
11,movies,Opening Weekend,actors,birth_day,0.204993,"vector_method=term_frequencies,similarity=cosine"
8,movies,Number of Theatres in Opening Weekend (US),actors,birth_month,0.200995,"vector_method=term_frequencies,similarity=cosine"
9,movies,Opening Weekend,actors,id,0.191291,"vector_method=term_frequencies,similarity=cosine"
10,movies,Opening Weekend,actors,birth_month,0.180216,"vector_method=term_frequencies,similarity=cosine"
1,movies,id,actors,age,0.174563,"vector_method=term_frequencies,similarity=cosine"
3,movies,id,actors,birth_day,0.169583,"vector_method=term_frequencies,similarity=cosine"


In [7]:
# Try different vector creation methods
vector_methods = ["binary_occurrence", "tfidf"]

for method in vector_methods:
    matcher = InstanceBasedSchemaMatcher(
        vector_creation_method=method,
        similarity_function="cosine",
        max_sample_size=500
    )
    
    mappings = matcher.match(
        source_dataset=movies_df,
        target_dataset=actors_df,
        threshold=0.1
    )
    
    print(f"\n{method.upper()} method found {len(mappings)} matches:")
    if not mappings.empty:
        for _, row in mappings.sort_values("score", ascending=False).head(3).iterrows():
            print(f"  {row['source_column']} <-> {row['target_column']} (score: {row['score']:.3f})")
    else:
        print("  No matches found above threshold")


BINARY_OCCURRENCE method found 11 matches:
  Audience Score <-> id (score: 0.752)
  Audience Score <-> age (score: 0.708)
  id <-> id (score: 0.306)

TFIDF method found 39 matches:
  Film <-> movie (score: 0.916)
  Rotten Tomatoes <-> id (score: 0.801)
  Foreign Gross <-> id (score: 0.777)


## Step 4: Duplicate-Based Schema Matching

Duplicate-based matching uses known record correspondences to determine schema mappings. This requires having correspondence data between datasets.

In [8]:
# Load academy awards and actors datasets (these have correspondences)
academy_path = root / "input" / "movies" / "entitymatching" / "data" / "academy_awards.xml"
correspondences_path = root / "input" / "movies" / "fusion" / "correspondences" / "academy_awards_2_actors_correspondences.csv"

# Load academy awards data
academy_df = load_xml(academy_path, name="academy_awards")
print(f"Academy awards shape: {academy_df.shape}")
print(f"Academy awards columns: {list(academy_df.columns)}")

# Load correspondences
correspondences_df = pd.read_csv(correspondences_path, header=None, 
                                names=["id1", "id2", "confidence"])
print(f"\nCorrespondences shape: {correspondences_df.shape}")
print("Sample correspondences:")
display(correspondences_df.head())

Academy awards shape: (4592, 7)
Academy awards columns: ['academy_awards_id', 'id', 'title', 'actor_name', 'date', 'director_name', 'oscar']

Correspondences shape: (150, 3)
Sample correspondences:


,id1,id2,confidence
0,academy_awards_4557,actors_1,1.0
1,academy_awards_4529,actors_2,1.0
2,academy_awards_4500,actors_3,1.0
3,academy_awards_4475,actors_4,1.0
4,academy_awards_4446,actors_5,1.0


In [9]:
# Preview the datasets for duplicate-based matching
print("=== Academy Awards Sample ===")
display(academy_df.head(3))

print("\n=== Actors Sample ===")
display(actors_df.head(3))

=== Academy Awards Sample ===


,academy_awards_id,id,title,actor_name,date,director_name,oscar
0,academy_awards-0000,academy_awards_1,Biutiful,Javier Bardem,2010-01-01,NaN,NaN
1,academy_awards-0001,academy_awards_2,True Grit,Jeff Bridges,2010-01-01,Joel Coen,NaN
2,academy_awards-0002,academy_awards_2,True Grit,Jeff Bridges,2010-01-01,Ethan Coen,NaN



=== Actors Sample ===


,actors_id,gender,id,year,name,movie,age,birthplace,birth_month,birth_day,birth_year
0,actors-0000,f,1,1929,Janet Gaynor,7th Heaven,22,Pennsylvania,10,6,1906
1,actors-0001,f,2,1930,Mary Pickford,Coquette,37,Canada,4,8,1892
2,actors-0002,f,3,1931,Norma Shearer,The Divorcee,28,Canada,8,10,1902


In [10]:
import re

def preprocess_correspondences(corr_df):
    """Fix correspondence IDs to match the actual dataset ID formats."""
    corr_fixed = corr_df.copy()
    
    def extract_actors_id(actor_id):
        """Convert 'actors_N' format to numeric N for actors dataset."""
        match = re.search(r'actors_(\d+)', str(actor_id))
        if match:
            return int(match.group(1))
        return actor_id
    
    corr_fixed['id2'] = corr_fixed['id2'].apply(extract_actors_id)
    return corr_fixed


# Fix the correspondences
correspondences_fixed = preprocess_correspondences(correspondences_df)

# Initialize duplicate-based matcher
duplicate_matcher = DuplicateBasedSchemaMatcher(
    vote_aggregation="majority",
    value_comparison="normalized",  # Normalize values before comparison
    min_votes=2,  # Require at least 2 votes for a correspondence
    ignore_zero_values=True
)

# Find schema correspondences using duplicate-based matching with FIXED correspondences
duplicate_mappings = duplicate_matcher.match(
    source_dataset=academy_df,
    target_dataset=actors_df,
    correspondences=correspondences_fixed,
    threshold=0.1  # Lower threshold to see more results
)

print(f"\nDuplicate-based matching found {len(duplicate_mappings)} correspondences:")
if not duplicate_mappings.empty:
    display(duplicate_mappings.sort_values("score", ascending=False))
else:
    print("No correspondences found above threshold.")


Duplicate-based matching found 2 correspondences:


,source_dataset,source_column,target_dataset,target_column,score,notes
0,academy_awards,title,actors,movie,1.000,"votes=40,method=duplicate_based"
1,academy_awards,actor_name,actors,name,0.625,"votes=25,method=duplicate_based"


In [11]:
# Explore fuzzy duplicate-based schema matching capabilities

print("=== Fuzzy Duplicate-Based Schema Matching ===")
print("This approach finds schema correspondences by analyzing how similar values")
print("appear across datasets using fuzzy string matching techniques.")

# Initialize fuzzy duplicate-based matcher with different similarity functions
similarity_functions = ["jaro_winkler", "levenshtein", "jaccard"]

for sim_func in similarity_functions:
    print(f"\n--- Using {sim_func.upper()} similarity ---")
    
    fuzzy_matcher = DuplicateBasedSchemaMatcher(
        vote_aggregation="majority",
        value_comparison="fuzzy",
        similarity_function=sim_func,
        similarity_threshold=0.7,  # Require 70% similarity for fuzzy matches
        min_votes=1,  # Allow single-vote correspondences for exploration
        ignore_zero_values=True
    )
    
    fuzzy_mappings = fuzzy_matcher.match(
        source_dataset=academy_df,
        target_dataset=actors_df,
        correspondences=correspondences_fixed,
        threshold=0.1
    )
    
    print(f"Found {len(fuzzy_mappings)} schema correspondences:")
    if not fuzzy_mappings.empty:
        for _, row in fuzzy_mappings.sort_values("score", ascending=False).iterrows():
            votes = row['notes'].split('votes=')[1].split(',')[0] if 'votes=' in row['notes'] else 'N/A'
            print(f"  {row['source_column']} → {row['target_column']} (score: {row['score']:.3f}, votes: {votes})")
    else:
        print("  No correspondences above threshold")

# Compare fuzzy vs normalized value comparison
print(f"\n=== Fuzzy vs Normalized Comparison ===")

# Normalized (exact) matching
normalized_matcher = DuplicateBasedSchemaMatcher(
    vote_aggregation="majority", 
    value_comparison="normalized",
    min_votes=1,
    ignore_zero_values=True
)

normalized_mappings = normalized_matcher.match(
    source_dataset=academy_df,
    target_dataset=actors_df, 
    correspondences=correspondences_fixed,
    threshold=0.1
)

# Fuzzy matching with jaro_winkler
fuzzy_matcher = DuplicateBasedSchemaMatcher(
    vote_aggregation="majority",
    value_comparison="fuzzy", 
    similarity_function="jaro_winkler",
    similarity_threshold=0.8,
    min_votes=1,
    ignore_zero_values=True
)

fuzzy_mappings = fuzzy_matcher.match(
    source_dataset=academy_df,
    target_dataset=actors_df,
    correspondences=correspondences_fixed,
    threshold=0.1
)

print(f"Normalized matching: {len(normalized_mappings)} correspondences")
print(f"Fuzzy matching: {len(fuzzy_mappings)} correspondences")

# Show detailed comparison
if not normalized_mappings.empty or not fuzzy_mappings.empty:
    print("\nDetailed Results:")
    
    if not normalized_mappings.empty:
        print("Normalized matches:")
        display(normalized_mappings[['source_column', 'target_column', 'score', 'notes']].sort_values('score', ascending=False))
    
    if not fuzzy_mappings.empty:
        print("Fuzzy matches:")
        display(fuzzy_mappings[['source_column', 'target_column', 'score', 'notes']].sort_values('score', ascending=False))

=== Fuzzy Duplicate-Based Schema Matching ===
This approach finds schema correspondences by analyzing how similar values
appear across datasets using fuzzy string matching techniques.

--- Using JARO_WINKLER similarity ---
Found 7 schema correspondences:
  title → movie (score: 1.000, votes: 41)
  actor_name → name (score: 0.634, votes: 26)
  date → birth_month (score: 0.585, votes: 24)
  date → birth_day (score: 0.488, votes: 20)
  date → birth_year (score: 0.293, votes: 12)
  date → id (score: 0.171, votes: 7)
  date → year (score: 0.122, votes: 5)

--- Using LEVENSHTEIN similarity ---
Found 2 schema correspondences:
  title → movie (score: 1.000, votes: 41)
  actor_name → name (score: 0.634, votes: 26)

--- Using JACCARD similarity ---
Found 2 schema correspondences:
  title → movie (score: 1.000, votes: 41)
  actor_name → name (score: 0.634, votes: 26)

=== Fuzzy vs Normalized Comparison ===
Normalized matching: 2 correspondences
Fuzzy matching: 3 correspondences

Detailed Results:

,source_column,target_column,score,notes
0,title,movie,1.000,"votes=40,method=duplicate_based"
1,actor_name,name,0.625,"votes=25,method=duplicate_based"


Fuzzy matches:


,source_column,target_column,score,notes
0,title,movie,1.000000,"votes=41,method=duplicate_based"
1,actor_name,name,0.634146,"votes=26,method=duplicate_based"
2,date,year,0.121951,"votes=5,method=duplicate_based"


## Step 5: Schema Mapping Evaluation

Evaluate the quality of schema mappings using precision, recall, and F1 scores. We'll create a small evaluation set for demonstration.

In [12]:
# Create a simple evaluation set (ground truth)
# In practice, this would come from domain experts or previous annotations
evaluation_set = pd.DataFrame([
    {"source_dataset": "movies", "source_column": "Film", 
     "target_dataset": "actors", "target_column": "movie", "label": True},
    {"source_dataset": "movies", "source_column": "year", 
     "target_dataset": "actors", "target_column": "year", "label": True},
    {"source_dataset": "movies", "source_column": "id", 
     "target_dataset": "actors", "target_column": "id", "label": True},
    {"source_dataset": "movies", "source_column": "Genre", 
     "target_dataset": "actors", "target_column": "name", "label": False},  # Negative example
])

print("Evaluation set:")
display(evaluation_set)

Evaluation set:


,source_dataset,source_column,target_dataset,target_column,label
0,movies,Film,actors,movie,True
1,movies,year,actors,year,True
2,movies,id,actors,id,True
3,movies,Genre,actors,name,False


In [13]:
# Evaluate label-based matching
evaluator = SchemaMappingEvaluator()

if not label_mappings.empty:
    label_metrics = evaluator.evaluate(
        corr=label_mappings,
        evaluation_set=evaluation_set,
        threshold=0.3,
        complete=False  # Not a complete evaluation set
    )
    
    print("Label-based matching evaluation:")
    for metric, value in label_metrics.items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.3f}")
        else:
            print(f"  {metric}: {value}")
else:
    print("No label-based mappings to evaluate.")

Label-based matching evaluation:
  precision: 1.000
  recall: 0.667
  f1: 0.800
  correct: 2
  matched: 2
  correct_total: 3
  missing: 1


In [14]:
# Sweep different thresholds for label-based matching
if not label_mappings.empty:
    threshold_results = evaluator.sweep_thresholds(
        corr=label_mappings,
        evaluation_set=evaluation_set,
        thresholds=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        complete=False
    )
    
    print("\nThreshold sweep results:")
    display(threshold_results[["threshold", "precision", "recall", "f1", "correct", "matched"]])
else:
    print("No mappings available for threshold sweep.")


Threshold sweep results:


,threshold,precision,recall,f1,correct,matched
0,0.1,1.0,0.666667,0.8,2,2
1,0.2,1.0,0.666667,0.8,2,2
2,0.3,1.0,0.666667,0.8,2,2
3,0.4,1.0,0.666667,0.8,2,2
4,0.5,1.0,0.666667,0.8,2,2
5,0.6,1.0,0.666667,0.8,2,2
6,0.7,1.0,0.666667,0.8,2,2
7,0.8,1.0,0.666667,0.8,2,2
8,0.9,1.0,0.666667,0.8,2,2


## Step 6: Data Translation

Apply schema mappings to transform DataFrames by renaming columns according to the discovered correspondences.

In [15]:
# Use the exact duplicate-based matches found earlier
# These are high-quality mappings based on actual record correspondences
print("=== Using Exact Duplicate-Based Schema Mappings ===")

print(f"Found {len(duplicate_mappings)} duplicate-based mappings:")
display(duplicate_mappings[["source_column", "target_column", "score", "notes"]])
mappings = duplicate_mappings

print(f"\nFinal mapping for translation: {len(mappings)} correspondences")

=== Using Exact Duplicate-Based Schema Mappings ===
Found 2 duplicate-based mappings:


,source_column,target_column,score,notes
0,title,movie,1.000,"votes=40,method=duplicate_based"
1,actor_name,name,0.625,"votes=25,method=duplicate_based"



Final mapping for translation: 2 correspondences


In [16]:
# Initialize the mapping translator
translator = MappingTranslator(strategy="rename")

# Apply the duplicate-based schema mapping to transform the academy_awards dataset
# Note: We're using academy_awards as source since that's what duplicate_mappings are based on
transformed_academy = translator.translate(academy_df, mappings)

print(f"Original academy_awards columns: {list(academy_df.columns)}")
print(f"Transformed academy_awards columns: {list(transformed_academy.columns)}")

# Show the transformation
renamed_cols = [col for col in transformed_academy.columns if col not in academy_df.columns]
if renamed_cols:
    print(f"\nRenamed columns: {renamed_cols}")
else:
    print(f"\nNo columns were renamed (mappings may target same column names)")

# Show which columns were mapped
mapped_source_cols = mappings['source_column'].tolist()
mapped_target_cols = mappings['target_column'].tolist()
print(f"Mapped: {dict(zip(mapped_source_cols, mapped_target_cols))}")

Original academy_awards columns: ['academy_awards_id', 'id', 'title', 'actor_name', 'date', 'director_name', 'oscar']
Transformed academy_awards columns: ['academy_awards_id', 'id', 'movie', 'name', 'date', 'director_name', 'oscar']

Renamed columns: ['movie', 'name']
Mapped: {'title': 'movie', 'actor_name': 'name'}


In [17]:
# Compare before and after transformation

print("=== Original Academy Awards Data Sample ===")
display(academy_df[mapped_source_cols].head(3))

print("\n=== Transformed Academy Awards Data Sample ===")  
display(transformed_academy[mapped_target_cols].head(3))

# Check provenance tracking
print("\n=== Provenance Information ===")
print(f"Dataset attrs keys: {list(transformed_academy.attrs.keys())}")
if 'provenance' in transformed_academy.attrs:
    print(f"Provenance entries: {len(transformed_academy.attrs['provenance'])}")
    print(f"Latest operation: {transformed_academy.attrs['provenance'][-1]['op']}")
else:
    print("No provenance tracking available")

=== Original Academy Awards Data Sample ===


,title,actor_name
0,Biutiful,Javier Bardem
1,True Grit,Jeff Bridges
2,True Grit,Jeff Bridges



=== Transformed Academy Awards Data Sample ===


,movie,name
0,Biutiful,Javier Bardem
1,True Grit,Jeff Bridges
2,True Grit,Jeff Bridges



=== Provenance Information ===
Dataset attrs keys: ['dataset_name', 'provenance']
Provenance entries: 2
Latest operation: schema_translate


## Step 7: Complete End-to-End Workflow

Demonstrate a complete schema matching and translation pipeline combining multiple techniques.

In [18]:
def complete_schema_alignment_workflow(source_df, target_df, correspondences=None):
    """Complete schema alignment workflow combining multiple matching techniques."""
    
    results = {"label": None, "instance": None, "duplicate": None}
    
    # 1. Label-based matching
    print("Step 1: Label-based matching...")
    label_matcher = LabelBasedSchemaMatcher(
        similarity_function="jaro_winkler",
        preprocess=str.lower,
        tokenize=True
    )
    
    results["label"] = label_matcher.match(
        source_dataset=source_df,
        target_dataset=target_df,
        threshold=0.4
    )
    print(f"  Found {len(results['label'])} label-based correspondences")
    
    # 2. Instance-based matching
    print("\nStep 2: Instance-based matching...")
    instance_matcher = InstanceBasedSchemaMatcher(
        vector_creation_method="term_frequencies",
        similarity_function="cosine",
        max_sample_size=300
    )
    
    results["instance"] = instance_matcher.match(
        source_dataset=source_df,
        target_dataset=target_df,
        threshold=0.15
    )
    print(f"  Found {len(results['instance'])} instance-based correspondences")
    
    # 3. Duplicate-based matching (if correspondences provided)
    if correspondences is not None and not correspondences.empty:
        print("\nStep 3: Duplicate-based matching...")
        duplicate_matcher = DuplicateBasedSchemaMatcher(
            vote_aggregation="majority",
            value_comparison="normalized",
            min_votes=1
        )
        
        results["duplicate"] = duplicate_matcher.match(
            source_dataset=source_df,
            target_dataset=target_df,
            correspondences=correspondences,
            threshold=0.05
        )
        print(f"  Found {len(results['duplicate'])} duplicate-based correspondences")
    
    # 4. Combine results (simple approach: take highest scoring mappings)
    print("\nStep 4: Combining results...")
    all_mappings = []
    for method, mappings in results.items():
        if mappings is not None and not mappings.empty:
            mappings = mappings.copy()
            mappings["method"] = method
            all_mappings.append(mappings)
    
    if all_mappings:
        combined_mappings = pd.concat(all_mappings, ignore_index=True)
        # Remove duplicates by keeping highest scoring mapping for each column pair
        combined_mappings = combined_mappings.sort_values("score", ascending=False).drop_duplicates(
            subset=["source_column", "target_column"], keep="first"
        )
        print(f"  Combined to {len(combined_mappings)} unique correspondences")
    else:
        combined_mappings = pd.DataFrame()
        print("  No mappings found")
    
    return results, combined_mappings

# Run complete workflow on academy -> actors with FIXED correspondences
print("=== Complete Schema Alignment Workflow ===")
print("Using academy_awards -> actors datasets with fixed correspondences")

workflow_results, final_mappings = complete_schema_alignment_workflow(
    source_df=academy_df,
    target_df=actors_df,
    correspondences=correspondences_fixed  # Use the fixed correspondences
)

print("\n=== Final Combined Mappings ===")
if not final_mappings.empty:
    display(final_mappings.sort_values("score", ascending=False)[["source_column", "target_column", "score", "method"]])
else:
    print("No final mappings generated.")
    
# Also run on the original movies -> actors comparison for comparison
print("\n" + "="*60)
print("=== Comparison: Movies -> Actors (without correspondences) ===")

workflow_results_movies, final_mappings_movies = complete_schema_alignment_workflow(
    source_df=movies_df,
    target_df=actors_df
)

print("\n=== Movies -> Actors Final Mappings ===")
if not final_mappings_movies.empty:
    display(final_mappings_movies.sort_values("score", ascending=False)[["source_column", "target_column", "score", "method"]])
else:
    print("No final mappings generated.")

=== Complete Schema Alignment Workflow ===
Using academy_awards -> actors datasets with fixed correspondences
Step 1: Label-based matching...
  Found 1 label-based correspondences

Step 2: Instance-based matching...
  Found 0 instance-based correspondences

Step 3: Duplicate-based matching...
  Found 2 duplicate-based correspondences

Step 4: Combining results...
  Combined to 3 unique correspondences

=== Final Combined Mappings ===


,source_column,target_column,score,method
0,id,id,1.000,label
1,title,movie,1.000,duplicate
2,actor_name,name,0.625,duplicate



=== Comparison: Movies -> Actors (without correspondences) ===
Step 1: Label-based matching...
  Found 2 label-based correspondences

Step 2: Instance-based matching...
  Found 10 instance-based correspondences

Step 4: Combining results...
  Combined to 11 unique correspondences

=== Movies -> Actors Final Mappings ===


,source_column,target_column,score,method
0,id,id,1.000000,label
1,year,year,1.000000,label
4,Film,movie,0.896943,instance
5,Audience Score,id,0.641639,instance
6,Audience Score,age,0.438684,instance
7,Story,movie,0.187786,instance
8,Number of Theatres in Opening Weekend (US),birth_month,0.182230,instance
9,Opening Weekend,id,0.164201,instance
3,id,birth_day,0.157181,instance
11,Opening Weekend,birth_day,0.154415,instance


In [19]:
# Apply the best mappings using data translation
if not final_mappings.empty:
    # Filter to high-confidence mappings
    high_confidence_mappings = final_mappings[final_mappings["score"] >= 0.5].copy()
    
    if not high_confidence_mappings.empty:
        print(f"Applying {len(high_confidence_mappings)} high-confidence mappings...")
        
        # Apply translation
        translator = MappingTranslator()
        aligned_movies = translator.translate(movies_df, high_confidence_mappings)
        
        print("\n=== Schema Alignment Results ===")
        print(f"Original movies columns: {len(movies_df.columns)}")
        print(f"Aligned movies columns: {len(aligned_movies.columns)}")
        
        # Show some aligned columns
        aligned_cols = [col for col in aligned_movies.columns if col not in movies_df.columns]
        if aligned_cols:
            print(f"New aligned columns: {aligned_cols}")
            sample_cols = (aligned_cols + [col for col in movies_df.columns if col not in high_confidence_mappings['source_column'].values])[:6]
            display(aligned_movies[sample_cols].head(3))
    else:
        print("No high-confidence mappings available for translation.")
else:
    print("No mappings available for translation.")

Applying 3 high-confidence mappings...

=== Schema Alignment Results ===
Original movies columns: 23
Aligned movies columns: 23


## Summary and Output

This notebook demonstrated the complete schema matching and data translation workflow in PyDI:

1. **Label-based matching**: Used string similarity on column names
2. **Instance-based matching**: Analyzed value distributions and content
3. **Duplicate-based matching**: Leveraged known record correspondences
4. **Evaluation**: Assessed matching quality with precision/recall metrics
5. **Translation**: Applied mappings to transform DataFrames
6. **End-to-end workflow**: Combined techniques for robust schema alignment

### Key Features Demonstrated:
- Multiple similarity functions (Jaccard, Levenshtein, Jaro-Winkler, Cosine)
- Different vector creation methods (term frequencies, binary occurrence, TF-IDF)
- Configurable thresholds and parameters
- Provenance tracking throughout the pipeline
- Evaluation and quality assessment tools
- Practical combination of multiple matching strategies

In [20]:
# Save results to output directory
output_dir = root / "output" / "examples" / "schemamatching"
output_dir.mkdir(parents=True, exist_ok=True)

# Save mappings if they exist
if not final_mappings.empty:
    mappings_path = output_dir / "schema_mappings.csv"
    final_mappings.to_csv(mappings_path, index=False)
    print(f"Saved schema mappings to: {mappings_path}")

# Save aligned dataset if it exists
if 'aligned_movies' in locals() and aligned_movies is not None:
    aligned_path = output_dir / "aligned_movies.csv"
    aligned_movies.to_csv(aligned_path, index=False)
    print(f"Saved aligned dataset to: {aligned_path}")

print(f"\nExample completed! Check {output_dir} for outputs.")

Saved schema mappings to: c:\Users\Ralph\dev\pydi\output\examples\schemamatching\schema_mappings.csv
Saved aligned dataset to: c:\Users\Ralph\dev\pydi\output\examples\schemamatching\aligned_movies.csv

Example completed! Check c:\Users\Ralph\dev\pydi\output\examples\schemamatching for outputs.
